In [ ]:
import py2neo
import tqdm
import pandas as pd

from tqdm import tqdm
from py2neo import Graph,Node,Relationship,NodeMatcher

In [ ]:
# pandas-style!
def color_negative_red(val):
    color = 'red' if val < 0 else 'black' 
    return 'color: %s' % color 
# df.style.applymap(color_negative_red)

In [ ]:
# pandas-num-option!
pd.options.display.max_columns = 50  # None -> No Restrictions
pd.options.display.max_rows = 200    # None -> Be careful with this
pd.options.display.max_colwidth = 100
pd.options.display.precision = 3

In [ ]:
trafficGraph = Graph('http://hapark02:3447',user = 'neo4j', password = '123')

In [ ]:
nodeDF = pd.read_csv('/usr/local/codeData/jupyterData/jupyter-bd/graph-algorithms/data/transport-nodes.csv')

In [ ]:
relationDF = pd.read_csv('/usr/local/codeData/jupyterData/jupyter-bd/graph-algorithms/data/transport-relationships.csv')

In [ ]:
nodeDF.head()

In [ ]:
# pandas-table-style!
relationDF.head(10).style.set_table_styles(
[{'selector': 'tr:nth-of-type(odd)','props': [('background', 'eee')]},
 {'selector': 'tr:nth-of-type(even)','props': [('background', 'white')]},
 {'selector': 'th','props': [('background', '#606060'), 
                             ('color', 'white'),
                             ('font-family', 'verdana')]},
 {'selector': 'td','props': [('font-family', 'verdana')]}
])

In [ ]:
relationDF

In [ ]:
for i, row in tqdm(nodeDF.iterrows()):
    
    transport_node = Node("transport_node",name=row[nodeDF.columns[0]])
    
    trafficGraph.merge(transport_node, "transport_node", "name")
    
    for j in range(len(list(row))-1):
        propertyKey = nodeDF.columns[j+1]
        propertyValue = row[nodeDF.columns[j+1]]
        transport_node.update({propertyKey:propertyValue})
        
    propertyKey = None
    propertyValue = None
        
    trafficGraph.push(transport_node)
    

In [ ]:
for i, row in tqdm(relationDF.iterrows()):
    
    startNodeName, endNodeName, relationType = [row[j] for j in range(3)]
    startNode = trafficGraph.nodes.match("transport_node", name = startNodeName).first()
    endNode = trafficGraph.nodes.match("transport_node", name = endNodeName).first()
    transport_relationship = Relationship(startNode,relationType,endNode)
    
    trafficGraph.create(transport_relationship)
    
    for j in row.index[3:]:
        transport_relationship.update({j:row[j]})
    
    trafficGraph.push(transport_relationship)
    